# Notebook 001: Creator Content Engine (Hooks + Captions + Hashtags)

## What you'll build
A simple **creator/marketer content pack generator** that turns one idea into:
- **10 hooks** (short-video openers)
- **10 captions** (platform-ready)
- **15 hashtags** (relevant + safe)
- **1 video angle summary** (what to show visually)

You can run this notebook:
- In **Google Colab** (fastest)
- Locally in **JupyterLab**

## Providers supported
- **OpenAI** (API key required)
- **Gemini** (API key required)
- **Ollama** (local model, no key)

Security note: API keys are secrets. Do not paste them into cells as plain text. We use hidden input.


### Privacy Note
This notebook does not track you.
Links are counted anonymously to understand what content is useful.


## 1) Runtime check: Colab vs Local
This helps us choose friendlier defaults.


## 2) Install dependencies
If running locally, you can also install via `pip install -r requirements.txt`.


## 3) Choose your provider
Set `PROVIDER` to one of:
- `"openai"`
- `"gemini"`
- `"ollama"`

Defaults:
- If you have an API key, start with OpenAI or Gemini.
- If you want free local inference, use Ollama (requires local install).


In [ ]:
import sys

IN_COLAB = 'google.colab' in sys.modules
print('✓ Running in Google Colab' if IN_COLAB else '✓ Running locally (Jupyter)')


## 4) Secure API key entry (hidden)
We use hidden prompts so the key is not displayed.

- OpenAI uses `OPENAI_API_KEY`.
- Gemini uses `GEMINI_API_KEY`.
- Ollama needs no key.


In [ ]:
import sys

def pip_install(pkgs: str) -> None:
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '-U'] + pkgs.split())

# Core utilities
pip_install('requests')

# Provider SDKs (install both; we will only use the one you select)
pip_install('openai')
pip_install('google-genai')

print('✓ Dependencies installed')


## 5) Unified LLM call helper
This gives us one function: `llm(prompt)` regardless of provider.


In [ ]:
PROVIDER = 'openai'  # 'openai' | 'gemini' | 'ollama'

# Model defaults (you can change these)
OPENAI_MODEL = 'gpt-4.1-mini'
GEMINI_MODEL = 'gemini-2.0-flash'
OLLAMA_MODEL = 'llama3.1'  # example; must exist locally

print('✓ Provider:', PROVIDER)


## 6) Connection test
If this succeeds, your provider is set up correctly.


In [ ]:
import os
from getpass import getpass

def ensure_key(env_name: str) -> None:
    if os.getenv(env_name):
        print(f'✓ {env_name} is set (hidden)')
        return
    key = getpass(f'Enter {env_name} (input hidden): ')
    if not key or not key.strip():
        raise ValueError(f'{env_name} is required for this provider.')
    os.environ[env_name] = key.strip()
    print(f'✓ {env_name} captured (not printed)')

if PROVIDER == 'openai':
    ensure_key('OPENAI_API_KEY')
elif PROVIDER == 'gemini':
    ensure_key('GEMINI_API_KEY')
elif PROVIDER == 'ollama':
    print('✓ Ollama selected (no API key needed)')
else:
    raise ValueError('Invalid PROVIDER. Use: openai | gemini | ollama')


## 7) Use case: generate a content pack
Fill in the inputs below. Keep them short.

Tip: For short-form video, hooks should be punchy and visual.


In [ ]:
import json
import requests

def llm(prompt: str) -> str:
    prompt = prompt.strip()
    if PROVIDER == 'openai':
        from openai import OpenAI
        client = OpenAI()
        resp = client.responses.create(
            model=OPENAI_MODEL,
            input=prompt
        )
        # Responses API returns output items; simplest: grab output_text
        return resp.output_text

    if PROVIDER == 'gemini':
        from google import genai
        client = genai.Client(api_key=os.environ.get('GEMINI_API_KEY'))
        resp = client.models.generate_content(
            model=GEMINI_MODEL,
            contents=prompt
        )
        return resp.text

    if PROVIDER == 'ollama':
        url = 'http://localhost:11434/api/generate'
        payload = {
            'model': OLLAMA_MODEL,
            'prompt': prompt,
            'stream': False
        }
        r = requests.post(url, json=payload, timeout=120)
        r.raise_for_status()
        data = r.json()
        return data.get('response', '')

    raise ValueError('Invalid PROVIDER')

print('✓ LLM helper ready')


### Generate (JSON output)
We ask for strict JSON so you can automate downstream.

If parsing fails, re-run and the prompt will force JSON again.


### Parse + save
We save to `outputs/001_content_pack.json` for reuse in your video generation pipeline.


In [ ]:
try:
    out = llm('Reply with exactly: OK')
    print('Model reply:', out.strip()[:50])
    print('✓ Connection test passed')
except Exception as e:
    print('✗ Connection test failed')
    raise


## 8) Next steps
- Duplicate this notebook into `002_...` and swap the **Use case** section.
- Keep the same provider/key scaffold.
- Your short videos can show:
  1) the hook concept
  2) the notebook cell running
  3) the JSON output that your pipeline uses.


In [ ]:
AUDIENCE = 'busy creators and marketers'
OFFER = 'free starter notebook that generates hooks, captions, hashtags'
NICHE = 'agentic AI for content creation'
TOPIC = 'Stop prompting. Start building a content system.'
TONE = 'confident, practical, not hype'
PLATFORM = 'TikTok'
CTA = 'Comment NOTEBOOK for the free download'

print('✓ Inputs set')


In [ ]:
PROMPT = f"""
You are a creator-marketer assistant.
Generate a content pack for short-form video.

Audience: {AUDIENCE}
Offer: {OFFER}
Niche: {NICHE}
Topic: {TOPIC}
Tone: {TONE}
Platform: {PLATFORM}
CTA: {CTA}

Rules:
- Hooks: 5 to 9 words each. Make them visual and specific.
- Captions: 1 to 2 sentences each. Simple language.
- Hashtags: relevant, non-spammy, avoid banned/unsafe themes.
- Angle summary: 1 short paragraph describing what visuals to show.
- Return STRICT JSON only. No markdown.

Return JSON schema:
{{
  "hooks": ["..."],
  "captions": ["..."],
  "hashtags": ["..."],
  "video_angle_summary": "..."
}}
""".strip()

raw = llm(PROMPT)
raw[:400]


In [ ]:
import os
from pathlib import Path

def parse_json_strict(s: str) -> dict:
    s = s.strip()
    # Basic cleanup if the model wraps JSON in text
    start = s.find('{')
    end = s.rfind('}')
    if start == -1 or end == -1:
        raise ValueError('No JSON object found in response.')
    return json.loads(s[start:end+1])

data = parse_json_strict(raw)

Path('outputs').mkdir(exist_ok=True)
out_path = Path('outputs/001_content_pack.json')
out_path.write_text(json.dumps(data, indent=2, ensure_ascii=False), encoding='utf-8')

print('✓ Saved:', out_path)
print('Hooks sample:', data['hooks'][:2])


## ✅ You're set up!

If you saw your content pack saved above, you've successfully generated hooks, captions, hashtags, and video angles!

**👉 Next steps to scale your content:**

1. **[Notebooks as Templates](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/ai-literacy/06_notebooks_as_templates.ipynb)** ← Learn to duplicate and reuse this notebook
2. **[What You Can Build](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/ai-literacy/07_what_you_can_build.ipynb)** ← See more content generation ideas
3. **[Start Here (Complete Guide)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/ai-literacy/00_start_here_jupyter_colab_models.ipynb)** ← Full beginner guide

**Want to learn more?**
- **[Colab vs Local](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/ai-literacy/01_google_colab_vs_local.ipynb)** ← Choose your environment
- **[What is a Notebook?](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/ai-literacy/02_what_is_jupyter_notebook.ipynb)** ← Learn the basics
- **[Secure API Keys](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/ai-literacy/04_secure_api_key_handling.ipynb)** ← Handle keys safely

**Or explore the full repo:**
- **[View on GitHub](https://github.com/YOUR_USERNAME/YOUR_REPO)** ← See all notebooks
- **[Main README](https://github.com/YOUR_USERNAME/YOUR_REPO/blob/main/README.md)** ← Full documentation
- **[Template Scaffold](../notebooks/_template_notebook_scaffold.md)** ← Create your own notebooks

---

**One click. No signup required. Just open and run!** 🚀
